# ROS2 bag file converter

## Imports and variables

In [ ]:
import os
from pathlib import Path

from rosbags.dataframe import get_dataframe
from rosbags.highlevel import AnyReader
from rosbags.typesys import get_types_from_msg, register_types

workspace_dir='/home/hugobaptista/ros2/ccpm'
bag_dir='IMU-rosbag_11032024'
save_dir='IMU-data'

# Add all topic message types to the reader's registered types
add_types = {}
for filename in os.listdir(f'{workspace_dir}/src/ccpm_msgs/msg'):
    text = Path(f'{workspace_dir}/src/ccpm_msgs/msg/{filename}').read_text()
    name=f'ccpm_msgs/msg/{filename.split(".")[0]}'
    add_types.update(get_types_from_msg(text, name))
register_types(add_types)

## Convert from bag to Pandas dataframe

In [ ]:
def convert_index_to_timestamp_column(df):
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'timestamp'}, inplace=True)

topics={}
dataframes={}
with AnyReader([Path(f'{workspace_dir}/{bag_dir}')]) as reader:
    # The 'topics' dictionary's keys are the topics names and the values are their arguments (list)
    for connection in reader.connections:
        keys = []
        for field in reader.typestore.FIELDDEFS[connection.msgtype][1]:
            keys.append(field[0])
        topics[connection.topic]=keys

    # The 'dataframes' dictionary's keys are the topics names and the values are their pandas dataframe
    for topic in topics:
        dataframes[topic]=get_dataframe(reader, topic, topics.get(topic))
        convert_index_to_timestamp_column(dataframes[topic])

for topic in dataframes:
    print(f'Topic: {topic}\nDataframe:{dataframes[topic].head}\n-----------------------------------------\n')

## Save the Pandas dataframes as csv files

In [ ]:
for topic in dataframes:
    new_file=f'{workspace_dir}/{save_dir}/csv/{topic[1:].replace("/", "_")}.csv'
    dataframes[topic].to_csv(new_file, index=False)
    print(f'The topic {topic} was saved as {new_file}')